In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip uninstall -y typing # this should avoid  AttributeError: type object 'Callable' has no attribute '_abc_registry'

!pip install  "git+https://github.com/dreamquark-ai/tabnet.git@develop#egg=pytorch_tabnet" --upgrade

  Cloning https://github.com/dreamquark-ai/tabnet.git (to revision develop) to /tmp/pip-install-t4avepdg/pytorch-tabnet_9b3fd42ab59344f88378e575a1d78f69
  Running command git clone -q https://github.com/dreamquark-ai/tabnet.git /tmp/pip-install-t4avepdg/pytorch-tabnet_9b3fd42ab59344f88378e575a1d78f69
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done


In [ ]:
# import os
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
from pytorch_tabnet.multitask import TabNetMultiTaskClassifier

import torch
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score, log_loss

import pandas as pd
import numpy as np
np.random.seed(0)

from tqdm.notebook import tqdm

import os

from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
path = '/content/drive/MyDrive/data/'

In [ ]:
import tensorflow as tf

import pandas as pd
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

In [ ]:
train = pd.read_csv(path+'train.csv')
train.shape

(26457, 20)

In [ ]:
test = pd.read_csv(path+'test.csv')
test.shape

(10000, 19)

In [ ]:
submission = pd.read_csv(path+'sample_submission.csv')

df=pd.concat([train, test], axis=0)
df.shape

(36457, 20)

In [ ]:
# df = pd.read_csv(path+'train.csv')
df.drop(['index','FLAG_MOBIL'], axis = 1, inplace = True)
df.head()

,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,work_phone,phone,email,occyp_type,family_size,begin_month,credit
0,F,N,N,0,202500.0,Commercial associate,Higher education,Married,Municipal apartment,-13899,-4709,0,0,0,NaN,2.0,-6.0,1.0
1,F,N,Y,1,247500.0,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,-11380,-1540,0,0,1,Laborers,3.0,-5.0,1.0
2,M,Y,Y,0,450000.0,Working,Higher education,Married,House / apartment,-19087,-4434,0,1,0,Managers,2.0,-22.0,2.0
3,F,N,Y,0,202500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-15088,-2092,0,1,0,Sales staff,2.0,-37.0,0.0
4,F,Y,Y,0,157500.0,State servant,Higher education,Married,House / apartment,-15037,-2105,0,0,0,Managers,2.0,-26.0,2.0


In [ ]:
df.gender = df.gender.replace({'F' : 0, 'M' : 1})
df.car = df.car.replace({'N' : 0, 'Y' : 1})
df.reality = df.reality.replace({'N' : 0, 'Y' : 1})
df.occyp_type = df.occyp_type.fillna('NaN')
df.head(3)

,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,work_phone,phone,email,occyp_type,family_size,begin_month,credit
0,0,0,0,0,202500.0,Commercial associate,Higher education,Married,Municipal apartment,-13899,-4709,0,0,0,NaN,2.0,-6.0,1.0
1,0,0,1,1,247500.0,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,-11380,-1540,0,0,1,Laborers,3.0,-5.0,1.0
2,1,1,1,0,450000.0,Working,Higher education,Married,House / apartment,-19087,-4434,0,1,0,Managers,2.0,-22.0,2.0


In [ ]:
df['age'] = df.DAYS_BIRTH.apply(lambda x : -x // 365)
df.DAYS_EMPLOYED = (-1) * df.DAYS_EMPLOYED 
df.loc[(df.DAYS_EMPLOYED < 0), 'DAYS_EMPLOYED'] = 0
df.begin_month = (-1) * df.begin_month

In [ ]:
from sklearn import preprocessing
label_encoder=preprocessing.LabelEncoder()
df['income_type']=label_encoder.fit_transform(df['income_type'])
df['edu_type']=label_encoder.fit_transform(df['edu_type'])
df['family_type']=label_encoder.fit_transform(df['family_type'])
df['house_type']=label_encoder.fit_transform(df['house_type'])
df['income_total']=label_encoder.fit_transform(df['income_total'])
df['occyp_type']=label_encoder.fit_transform(df['occyp_type'])

In [ ]:
train=df[:len(df)-10000]
test=df[len(df)-10000:]

In [ ]:
train_x=train.drop('credit', axis=1)
train_y=train[['credit']]
test_x=test.drop('credit', axis=1)

In [ ]:
np.random.seed(42)
if "Set" not in train.columns:
    train["Set"] = np.random.choice(["train", "valid"], p =[.8, .2], size=(train.shape[0],))

train_indices = train_x[train.Set=="train"].index
valid_indices = train_x[train.Set=="valid"].index

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
nunique = train.nunique()
types = train.dtypes

categorical_columns = []
categorical_dims =  {}
for col in tqdm(train_x.columns):
    if types[col] == 'object' or nunique[col] < 200:
        print(col, train_x[col].nunique())
        l_enc = LabelEncoder()
        train_x[col] = train_x[col].fillna("NaN")
        train_x[col] = l_enc.fit_transform(train_x[col].values)
        try:
            test_x[col] = test_x[col].fillna("NaN")
            test_x[col] = l_enc.transform(test_x[col].values)
        except:
            print(f"Column {col} does not exist in test set")
        categorical_columns.append(col)
        categorical_dims[col] = len(l_enc.classes_)
    else:
        training_mean = train_x.loc[train_indices, col].mean()
        train.fillna(training_mean, inplace=True)
        test.fillna(training_mean, inplace=True)

  0%|          | 0/18 [00:00<?, ?it/s]

gender 2
car 2
reality 2
child_num 9
income_type 5
edu_type 5
family_type 5
house_type 6
work_phone 2
phone 2
email 2
occyp_type 19
family_size 10
begin_month 61
age 48
Column age does not exist in test set


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [ ]:
features = [ col for col in train_x.columns] 

cat_idxs = [ i for i, f in enumerate(features) if f in categorical_columns]

cat_dims = [ categorical_dims[f] for i, f in enumerate(features) if f in categorical_columns]

In [ ]:
print("features: ", features)
print("cat_idxs: ", cat_idxs)
print("cat_dims: ", cat_dims)

features:  ['gender', 'car', 'reality', 'child_num', 'income_total', 'income_type', 'edu_type', 'family_type', 'house_type', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'work_phone', 'phone', 'email', 'occyp_type', 'family_size', 'begin_month', 'age']
cat_idxs:  [0, 1, 2, 3, 5, 6, 7, 8, 11, 12, 13, 14, 15, 16, 17]
cat_dims:  [2, 2, 2, 9, 5, 5, 5, 6, 2, 2, 2, 19, 10, 61, 48]


In [ ]:
cat_dims[14] = 69

In [ ]:
print("features: ", features)
print("cat_idxs: ", cat_idxs)
print("cat_dims: ", cat_dims)

features:  ['gender', 'car', 'reality', 'child_num', 'income_total', 'income_type', 'edu_type', 'family_type', 'house_type', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'work_phone', 'phone', 'email', 'occyp_type', 'family_size', 'begin_month', 'age']
cat_idxs:  [0, 1, 2, 3, 5, 6, 7, 8, 11, 12, 13, 14, 15, 16, 17]
cat_dims:  [2, 2, 2, 9, 5, 5, 5, 6, 2, 2, 2, 19, 10, 61, 69]


In [ ]:
X_train = train_x[features].values[train_indices]
y_train = train_y.values[train_indices]

X_valid = train_x[features].values[valid_indices]
y_valid = train_y.values[valid_indices]

X_test = test_x[features].values

In [ ]:
clf = TabNetMultiTaskClassifier(n_steps=1,
                                cat_idxs=cat_idxs,
                                cat_dims=cat_dims,
                                cat_emb_dim=1,
                                optimizer_fn=torch.optim.Adam,
                                optimizer_params=dict(lr=2e-2),
                                scheduler_params={"step_size":50,
                                                  "gamma":0.9},
                                scheduler_fn=torch.optim.lr_scheduler.StepLR,
                                mask_type='entmax', 
                                lambda_sparse=0, 
                       
                      )

/usr/local/lib/python3.7/dist-packages/pytorch_tabnet/abstract_model.py:75: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


In [ ]:
max_epochs = 1000

clf.fit(
    X_train=X_train, y_train=y_train,
    max_epochs=max_epochs ,
    patience=50,
    batch_size=1024,
    virtual_batch_size=128,
    num_workers=1,
    drop_last=False,
)

/usr/local/lib/python3.7/dist-packages/pytorch_tabnet/abstract_model.py:651: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


epoch 0  | loss: 0.97508 |  0:00:00s
epoch 1  | loss: 0.88369 |  0:00:01s
epoch 2  | loss: 0.87472 |  0:00:03s
epoch 3  | loss: 0.86263 |  0:00:04s
epoch 4  | loss: 0.84834 |  0:00:05s
epoch 5  | loss: 0.82958 |  0:00:07s
epoch 6  | loss: 0.81765 |  0:00:08s
epoch 7  | loss: 0.81371 |  0:00:09s
epoch 8  | loss: 0.80995 |  0:00:11s
epoch 9  | loss: 0.80845 |  0:00:12s
epoch 10 | loss: 0.80481 |  0:00:13s
epoch 11 | loss: 0.80204 |  0:00:15s
epoch 12 | loss: 0.79979 |  0:00:16s
epoch 13 | loss: 0.8001  |  0:00:18s
epoch 14 | loss: 0.79754 |  0:00:19s
epoch 15 | loss: 0.79419 |  0:00:21s
epoch 16 | loss: 0.7866  |  0:00:22s
epoch 17 | loss: 0.78341 |  0:00:24s
epoch 18 | loss: 0.78145 |  0:00:25s
epoch 19 | loss: 0.77936 |  0:00:26s
epoch 20 | loss: 0.77772 |  0:00:26s
epoch 21 | loss: 0.77688 |  0:00:27s
epoch 22 | loss: 0.77637 |  0:00:28s
epoch 23 | loss: 0.7749  |  0:00:29s
epoch 24 | loss: 0.77265 |  0:00:29s
epoch 25 | loss: 0.7687  |  0:00:30s
epoch 26 | loss: 0.76893 |  0:00:31s
e

In [ ]:
preds_valid = clf.predict_proba(X_valid) 

In [ ]:
preds = clf.predict_proba(X_test)

In [ ]:
clf.feature_importances_

array([0.01736483, 0.11222949, 0.19512481, 0.00831699, 0.03476829,
       0.10981537, 0.0253694 , 0.01346673, 0.        , 0.04499628,
       0.03768193, 0.07158021, 0.11575516, 0.02105896, 0.07538192,
       0.04241471, 0.04056674, 0.03410817])

In [ ]:
i, j, k = [], [], []
for row in range(10000):
  i.append(preds[0][row][0])
  j.append(preds[0][row][1])
  k.append(preds[0][row][2])

len(i), len(j), len(k)

(10000, 10000, 10000)

In [ ]:
submission['0'] = i
submission['1'] = j
submission['2'] = k
submission.head(10)

,index,0,1,2
0,26457,0.103893,0.097366,0.798741
1,26458,0.126568,0.189666,0.683766
2,26459,0.203722,0.270263,0.526015
3,26460,0.105986,0.090246,0.803767
4,26461,0.114425,0.201896,0.683679
5,26462,0.109755,0.270388,0.619858
6,26463,0.179081,0.819500,0.001419
7,26464,0.142630,0.413380,0.443990
8,26465,0.137475,0.118921,0.743605
9,26466,0.156965,0.331052,0.511983


In [ ]:
submission.to_csv(path+"submisison_tabnet_4.csv",index=False)

In [ ]:
prediction = pd.read_csv(path+'submisison_tabnet_4.csv')

In [ ]:
prediction

,index,0,1,2
0,26457,0.103893,0.097366,0.798741
1,26458,0.126568,0.189666,0.683766
2,26459,0.203722,0.270263,0.526015
3,26460,0.105986,0.090246,0.803767
4,26461,0.114425,0.201896,0.683679
...,...,...,...,...
9995,36452,0.182772,0.225129,0.592099
9996,36453,0.154418,0.201444,0.644138
9997,36454,0.030608,0.112735,0.856657
9998,36455,0.335593,0.116986,0.547421
